# Deep Learning Playground

We are interested in the following features:

- H1: region-averaged time-series
- H2: region-level connectivities (from H1, optional: triu-k1)
- H3: network-averaged time-series (from H1)
- H4: network connectivity (from H3, optional: triu-k1)
- H5: networks connectivity (from H2, optional: triu-k0)

> Note that we are not going to take the upper triangular part of the connectivity matrix and full matrices are used instead.

In [ ]:
%reload_ext autoreload
%autoreload 2

import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import RichProgressBar

from src.acnets.deep import ACNetsDataModule, MultiHeadModel
from src.acnets.pipeline import Parcellation

In [ ]:
x1 = Parcellation(
    atlas_name='dosenbach2010',
    bids_dir='~/workspace/acnets/data/julia2018',
    fmriprep_bids_space='MNI152NLin2009cAsym',
    normalize=True,
).fit_transform(X=None)

In [ ]:
datamodule = ACNetsDataModule(atlas='dosenbach2010', kind='partial correlation', batch_size=8)
datamodule.setup()

n_regions = datamodule.train[0][0].shape[1]
n_networks = datamodule.train[0][2].shape[1]

# DEBUG single epoch, one train/test split
model = MultiHeadModel(n_regions, n_networks, n_embeddings=8)
trainer = pl.Trainer(accelerator='auto', max_epochs=1000,
                     log_every_n_steps=1, callbacks=[RichProgressBar()])
trainer.fit(model, datamodule)
trainer.test(model, datamodule=datamodule)

In [ ]:
from ray.train.lightning import (
    RayDDPStrategy,
    RayLightningEnvironment,
    RayTrainReportCallback,
    prepare_trainer,
)
from ray import tune
from ray.train.torch import TorchTrainer
from ray.train import RunConfig, ScalingConfig, CheckpointConfig

def train_func(config):
    datamodule = ACNetsDataModule(atlas=config['atlas'],
                                  kind=config['kind'],
                                  batch_size=config['batch_size'])
    datamodule.setup()

    n_regions = datamodule.train[0][0].shape[1]
    n_networks = datamodule.train[0][2].shape[1]
    model = MultiHeadModel(n_regions, n_networks, n_embeddings=config['n_embeddings'])

    trainer = pl.Trainer(
        strategy=RayDDPStrategy(),
        callbacks=[RayTrainReportCallback()],
        plugins=[RayLightningEnvironment()],
        enable_progress_bar=False,
        max_epochs=config['max_epochs'],
        log_every_n_steps=1
    )
    trainer = prepare_trainer(trainer)
    trainer.fit(model, datamodule=datamodule)
    trainer.test(model, datamodule=datamodule)

TorchTrainer(
    train_func,
    train_loop_config={
        'batch_size': tune.choice([32]),
        'atlas': tune.choice(['dosenbach2010']),
        'kind': tune.choice(['partial correlation']),
        'n_embeddings': tune.choice([8]),
        'max_epochs': 1,
    },
    run_config=RunConfig(
        name='acnets_multihead',
        storage_path='~/ray_results',
        checkpoint_config=CheckpointConfig(
            num_to_keep=1,
            checkpoint_score_attribute='val/accuracy',
            checkpoint_score_order='max',
        )
    ),
    scaling_config=ScalingConfig(num_workers=2, use_gpu=False)
).fit()

In [ ]:
atlases = [
    'dosenbach2010',
    'dosenbach2007',
    'difumo_64_2mm',
    'difumo_128_2mm',
    'difumo_256_2mm',
    'difumo_512_2mm',
    'difumo_1024_2mm',
    'gordon2014_2mm',
    'cort-maxprob-thr25-2mm',
    'seitzman2018',
    'friedman2020',
]

kinds = [
    'correlation',
    'partial correlation',
    'tangent',
    'covariance',
    'precisions'
]